In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import toad
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题

# 加载训练集和测试集，合并一起处理

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
df = pd.concat([train,test],axis=0,ignore_index=True)
df.head(),df.shape
idx = train.index

# 分组处理特征

## 客户基础属性

### 填充sex，marriage_satatus，occupation，educate属性同时为空的行，填充值为0

In [3]:
df1_2 = df.copy()
df1_2_1 = df1_2[df1_2['sex'].isnull() & df1_2['marriage_satatus'].isnull() & df1_2['occupation'].isnull() & df1_2['educate'].isnull()]
df1_2.loc[df1_2_1.index,'sex':'educate'] = df1_2.loc[df1_2_1.index,'sex':'educate'].fillna(0)

### sex特征的异常值用0替代 

In [4]:
df2 = df1_2.copy()
for i in df2[(df2['sex'] < 0) | (df2['sex']>2)].index:
    df2.loc[i,'sex']= 0

### marriage_satatus特征异常值3，4，5，6，7全部设为0

In [5]:
df2_3 = df2.copy()
idx1= df2_3[(df2_3['marriage_satatus'] > 2) & (df2_3['marriage_satatus'] <8)].index
df2_3.loc[idx1,'marriage_satatus']= 0

### age特征不分箱，只将小于0和大于100设为-1

In [6]:
df3_2 = df2_3.copy()
idx2 = df3_2[(df3_2['age'] < 0) | (df3_2['age'] >100)].index
df3_2.loc[idx2,'age']=0

### occupation特征与educate特征同时为空的客户填充为0，剩下的用众数填充

In [7]:
df4_1 = df3_2.copy()
df4_1_1 = df4_1[df4_1['occupation'].isnull() & df4_1['educate'].isnull()]
df4_1['occupation'] = df4_1['occupation'].fillna(df4_1['occupation'].mode()[0])

### educate特征缺失值用0填充

In [8]:
df5_1 = df4_1.copy()
df5_1['educate'] = df5_1['educate'].fillna(0)

iv_info1 = toad.quality(df5_1.iloc[idx,:7],'flag', iv_only=True)
print(iv_info1)

df5_1.iloc[:,:7].isnull().sum()

                         iv  gini  entropy    unique
cust_id           24.797253   NaN      NaN  784000.0
occupation         0.881688   NaN      NaN      88.0
age                0.774163   NaN      NaN     101.0
marriage_satatus   0.395075   NaN      NaN       5.0
educate            0.173398   NaN      NaN      11.0
sex                0.089331   NaN      NaN       3.0


cust_id                  0
flag                340043
sex                      0
marriage_satatus         0
age                      0
occupation               0
educate                  0
dtype: int64

## 账户数量:'acctage','covaccvnumvm'

### 'acctage'和'covaccvnumvm'缺失值用-2填充

In [9]:
Num_accounts = train.columns[7:14].tolist()
Num_accounts_idx_flag = pd.Index(['flag'] +train.columns[7:14].tolist())
df6 = df5_1.copy()
for i in train.columns[7:14]:
    df6[i] = df6[i].fillna(-2)

In [10]:
iv_info2 = toad.quality(df6.loc[idx,Num_accounts_idx_flag],'flag', iv_only=True)
print(iv_info2)

                        iv  gini  entropy  unique
covaccvnumvm0024  2.778237   NaN      NaN   172.0
covaccvnumvm0012  2.743324   NaN      NaN   138.0
covaccvnumvm0006  2.702905   NaN      NaN   117.0
covaccvnumvm0003  2.670323   NaN      NaN   107.0
covaccvnumvm0002  2.653367   NaN      NaN   109.0
covaccvnumvm0001  2.635448   NaN      NaN   103.0
acctage           0.537352   NaN      NaN   615.0


### 'covaccvnumvm'特征只保留IV值最大的一个

In [11]:
drop_list = Num_accounts
drop_list.remove('covaccvnumvm0024')
drop_list.remove('acctage')
len(drop_list)

5

## 账户交易情况

### cfvfdpvnumvm,先用-1填充，再保留IV值最大的特征

### cfvdctrdamovm0001-0012,cfvdctrdvnumvm0001-0012,cfvcctrdamovm0001-0012,cfvcctrdvnumvm0001-0012，每组保留IV值最大的特征

In [12]:
df7 = df6.copy()
Account_transactions = train.columns[14:35].tolist()
train1 = train[Account_transactions]
col3 = (train1.shape[0]-train1.count())/train1.shape[0]#缺失率
Account_transactions_idx1 = col3[col3>0.8].index
Account_transactions_idx2 = col3[col3<0.8].index
Account_transactions1 = Account_transactions_idx1.tolist()
Account_transactions2 = Account_transactions_idx2.tolist()

In [13]:
df7[Account_transactions] = df7[Account_transactions].fillna(-1)
Account_transactions_idx1_flag = pd.Index(Account_transactions_idx1.tolist() + ['flag'])
iv_info4 = toad.quality(df7.loc[idx,Account_transactions_idx1_flag],'flag', iv_only=True)
print(iv_info4)

                        iv  gini  entropy  unique
cfvfdpvnumvm0012  5.512248   NaN      NaN   125.0
cfvfdpvnumvm0006  5.509688   NaN      NaN    88.0
cfvfdpvnumvm0003  5.508722   NaN      NaN    59.0
cfvfdpvnumvm0001  5.508182   NaN      NaN    33.0
cfvfdpvnumvm0002  5.507965   NaN      NaN    47.0


In [14]:
Account_transactions1_remove = Account_transactions1
Account_transactions1_remove.remove('cfvfdpvnumvm0012')

In [15]:
Account_transactions2.remove('cfvcctrdamovm0012')
Account_transactions2.remove('cfvdctrdamovm0012')
Account_transactions2.remove('cfvcctrdvnumvm0006')
Account_transactions2.remove('cfvdctrdvnumvm0012')

In [16]:
drop_list = drop_list + Account_transactions1_remove + Account_transactions2
len(drop_list)

21

In [18]:
df7.shape

(1124043, 339)

## 资产情况:'covassvhypvm','covassbalvmaxvm','covasshypvst2vm'

### 'covassvhypvm'和'covassbalvmaxvm'先用-1填充，再保留IV值最大的特征

In [17]:
df8 = df7.copy()
Assets_idx = train.columns[35:48]
Assets_idx1 = Assets_idx[:10]#'covassvhypvm'和'covassbalvmaxvm'的索引
df8[Assets_idx]=df8[Assets_idx].fillna(-1)

In [18]:
Assets_idx1_flag = pd.Index(Assets_idx1.tolist() + ['flag'])
iv_info5 = toad.quality(df8.loc[idx,Assets_idx1_flag],'flag', iv_only=True)
print(iv_info5)

                           iv  gini  entropy    unique
covassbalvmaxvm0001  5.284373   NaN      NaN  282047.0
covassbalvmaxvm0002  5.273959   NaN      NaN  288693.0
covassbalvmaxvm0003  5.273780   NaN      NaN  293013.0
covassvhypvm0001     5.248523   NaN      NaN  350873.0
covassbalvmaxvm0006  5.244686   NaN      NaN  302233.0
covassvhypvm0002     5.229474   NaN      NaN  361086.0
covassvhypvm0003     5.209058   NaN      NaN  369721.0
covassbalvmaxvm0012  5.190056   NaN      NaN  315034.0
covassvhypvm0006     5.176056   NaN      NaN  382413.0
covassvhypvm0012     5.131218   NaN      NaN  394783.0


In [19]:
Assets_idx1_remove = Assets_idx1.tolist()
Assets_idx1_remove.remove('covassbalvmaxvm0001')
Assets_idx1_remove.remove('covassvhypvm0001')

### covasshypvst2vmst2vm用-1填充，并保留

In [20]:
drop_list = drop_list + Assets_idx1_remove
len(drop_list)

29

## 贷款情况:cfvapplsuctimvm cfvreplreftimvm covreplsucmaxvm cfvalloantimvm

### 缺失率过大，全部丢弃

In [21]:
Loan_idx = train.columns[48:58]
drop_list = drop_list + Loan_idx.tolist()
len(drop_list)

39

## 渠道交易1

### 保留没有缺失值的特征,再丢掉其中相关性大的特征

In [22]:
Channel_transaction_idx = train.columns[58:166]

In [23]:
df8_1 = df8[Channel_transaction_idx]
col3 = (df8_1.shape[0]-df8_1.count())/df8_1.shape[0]
Channel_transaction_idx1 = col3[col3>0.7].index
Channel_transaction_idx2 = col3[col3<=0.7].index

In [24]:
df8[Channel_transaction_idx]=df8[Channel_transaction_idx].fillna(-1)

In [25]:
Channel_transaction2_remove = Channel_transaction_idx2.tolist()

In [26]:
Channel_transaction2_retain = ['cbvartavgvamovm0001','cbvartavgvnumvm0001','cbvartavgvnumvm0002',
                               'cbvartavgvnumvm0003','cbvartavgvnumvm0006','cbvartbhevamovm0003',
                               'cbvartbhevamovm0006','cbvartbhevamovm0012','cbvartbhevamovm0024',
                               'cbvartbhevnumvm0024','cbvartnumvst2vm0024','cbvartamovst2vm0024']

In [27]:
for i in Channel_transaction2_retain:
    Channel_transaction2_remove.remove(i)
len(Channel_transaction2_remove)

28

In [28]:
drop_list = drop_list + Channel_transaction_idx1.tolist() + Channel_transaction2_remove
len(drop_list)

135

## 渠道交易2

### 缺失率过大，全部丢弃

In [29]:
Channel_transaction2_idx = train.columns[314:330]
drop_list = drop_list + Channel_transaction2_idx.tolist()
len(drop_list)

151

## 渠道行为

### 保留其中IV值最高的一个特征

In [30]:
Channel_behavior_idx = train.columns[166:178]

In [31]:
df8[Channel_behavior_idx]=df8[Channel_behavior_idx].fillna(-1)

In [32]:
Channel_behavior_remove = Channel_behavior_idx.tolist()
Channel_behavior_remove.remove('cbvmobblogvnumvm0012')

In [33]:
drop_list = drop_list + Channel_behavior_remove
len(drop_list)

162

## 第三方交易:cbvalibhedctnumvm、cbvweccctavgamovm

### 缺失率过大，全部丢弃

In [34]:
Third_party_transactions_idx = train.columns[178:282]
drop_list = drop_list + Third_party_transactions_idx.tolist()
len(drop_list)

266

## 自助设备交易 cbvatmdepvamovm、cbvatmtrnivnumvm

### 缺失率过大，全部丢弃

In [35]:
Self_service_transaction_idx = train.columns[282:314]
drop_list = drop_list + Self_service_transaction_idx.tolist()
len(drop_list)

298

## 其他标识

### settime

In [36]:
df9 = df8.copy()
df9['settime'] = df9['settime'].astype('str')
df9['settime_month'] = df9['settime'].apply(lambda x: x[4:6])
df9['settime_month'] = df9['settime_month'].astype('int')

In [37]:
drop_list = drop_list + ['online_loans_flag', 'cbvwebloandamovm0006', 
                         'credit_card_flag','cbvcreditcardamovm0006', 
                         'pre_list_glag', 'apply_flag', 'aprv_status','merchant_falg','settime']
len(drop_list)

307

In [38]:
df9.drop(drop_list,axis  = 1,inplace = True)

In [39]:
df9.shape

(1124043, 33)

# 特征衍生

## 组内特征交叉

### 账户数量(时间差分与时间平均)

In [40]:
data1_1_diff1,data1_1_diff2,data1_1_diff3 = df['covaccvnumvm0024']-df['covaccvnumvm0001'],df['covaccvnumvm0012']-df['covaccvnumvm0001'],df['covaccvnumvm0006']-df['covaccvnumvm0001']

In [41]:
data1_1_diff = pd.concat([data1_1_diff1,data1_1_diff2,data1_1_diff3],axis=1)

In [42]:
data1_1_diff.columns=['covaccvnumvm24_01','covaccvnumvm12_01','covaccvnumvm06_01']

In [43]:
data1_1_diff['covaccvnumvm_avg'] = df.eval('(covaccvnumvm0024+covaccvnumvm0012+covaccvnumvm0006+covaccvnumvm0003+covaccvnumvm0002+covaccvnumvm0001)/6')

In [44]:
data1_1_diff['covaccvnumvm0024_avg'] = df['covaccvnumvm0024']- data1_1_diff['covaccvnumvm_avg']

In [45]:
data1_1_diff['covaccvnumvm0024_acctage'] = df['covaccvnumvm0024']- df['acctage']

In [46]:
data1_1_diff = data1_1_diff.fillna(-1)

### 账户交易情况(时间差分，时间平均，数量平均)

In [47]:
data2_1_diff = df['cfvfdpvnumvm0012'] - df['cfvfdpvnumvm0001']

In [48]:
data2_2_diff = df['cfvcctrdamovm0012'] - df['cfvcctrdamovm0001']

In [49]:
data2_3_diff = df['cfvdctrdamovm0012'] - df['cfvdctrdamovm0001']

In [50]:
data2_4_diff = df['cfvcctrdvnumvm0012'] - df['cfvcctrdvnumvm0001']

In [51]:
data2_5_diff = df['cfvdctrdvnumvm0012'] - df['cfvdctrdvnumvm0001']

In [52]:
data2_diff = pd.concat([data2_1_diff,data2_2_diff,data2_3_diff,data2_4_diff,data2_5_diff],axis=1)

In [53]:
data2_diff.columns=['cfvfdpvnumvm12_1','cfvcctrdamovm12_1','cfvdctrdamovm12_1',
                    'cfvcctrdvnumvm12_1','cfvdctrdvnumvm12_1']

In [54]:
data2_diff['cfvfdpvnumvm_avg'] = df.eval('(cfvfdpvnumvm0012+cfvfdpvnumvm0006+cfvfdpvnumvm0003+cfvfdpvnumvm0002+cfvfdpvnumvm0001)/5')

In [55]:
data2_diff['cfvcctrdamovm_avg'] = df.eval('(cfvcctrdamovm0012+cfvcctrdamovm0006+cfvcctrdamovm0003+cfvcctrdamovm0001)/4')

In [56]:
data2_diff['cfvcctrdvnumvm_avg'] = df.eval('(cfvcctrdvnumvm0012+cfvcctrdvnumvm0006+cfvcctrdvnumvm0003+cfvcctrdvnumvm0001)/4')

In [57]:
data2_diff['cfvdctrdamovm_avg'] = df.eval('( cfvdctrdamovm0012+ cfvdctrdamovm0006+ cfvdctrdamovm0003+ cfvdctrdamovm0001)/4')

In [58]:
data2_diff['cfvdctrdvnumvm_avg'] = df.eval('(cfvdctrdvnumvm0012+cfvdctrdvnumvm0006+cfvdctrdvnumvm0003+cfvdctrdvnumvm0001)/4')

In [59]:
data2_diff = data2_diff.fillna(-1)

In [60]:
data2_diff['trdamo0012'] = df['cfvcctrdamovm0012'] - df['cfvdctrdamovm0012']

In [61]:
data2_diff['trdvnum0012'] = df['cfvcctrdvnumvm0012'] - df['cfvdctrdvnumvm0012']

In [62]:
data2_diff['cfvcctrdavgvm0012'] = df['cfvcctrdamovm0012']/df['cfvcctrdvnumvm0012']

In [63]:
data2_diff['cfvdctrdavgvm0012'] = df['cfvdctrdamovm0012']/df['cfvdctrdvnumvm0012']

In [64]:
data2_diff = data2_diff.replace([np.inf, -np.inf,np.NaN], 0)

### 资产状况(时间平均)

In [65]:
data3_1_diff = df.eval('(covassvhypvm0012+covassvhypvm0006+covassvhypvm0003+covassvhypvm0002+covassvhypvm0001)/5')

In [66]:
data3_2_diff = df.eval('(covassbalvmaxvm0012+covassbalvmaxvm0006+covassbalvmaxvm0003+covassbalvmaxvm0002+covassbalvmaxvm0001)/5')

In [67]:
data3_3_diff = df.eval('(covasshypvst2vm0003+covasshypvst2vm0006+covasshypvst2vm0012)/3')

In [68]:
data3_diff = pd.concat([data3_1_diff,data3_2_diff,data3_3_diff],axis=1)

In [69]:
data3_diff.columns=['covassvhypvm_avg','covassbalvmaxvm_avg','covasshypvst2vm_avg']

In [70]:
data3_diff = data3_diff.fillna(-1)

In [85]:
df10 = df9.copy()

In [86]:
df10 = pd.concat([df10,data1_1_diff,data2_diff,data3_diff],axis=1)

In [87]:
df10.shape

(1124043, 56)

In [88]:
df10.to_csv('./df10.csv')

### 渠道交易1与渠道行为(时间差分，时间平均)

In [89]:
df10['cbvartbhevnumvm_avg'] = df.eval('(cbvartbhevnumvm0024+cbvartbhevnumvm0012+cbvartbhevnumvm0006+cbvartbhevnumvm0003+cbvartbhevnumvm0002+cbvartbhevnumvm0001)/6')

In [90]:
df10['cbvartbhevamovm_avg'] = df.eval('(cbvartbhevamovm0024+cbvartbhevamovm0012+cbvartbhevamovm0006+cbvartbhevamovm0003+cbvartbhevamovm0002+cbvartbhevamovm0001)/6')

In [91]:
df10['cbvartavgvnumvm_avg'] = df.eval('(cbvartavgvnumvm0024+cbvartavgvnumvm0012+cbvartavgvnumvm0006+cbvartavgvnumvm0003+cbvartavgvnumvm0002+cbvartavgvnumvm0001)/6')

In [92]:
df10['cbvartavgvamovm_avg']  = df.eval('(cbvartavgvamovm0024+cbvartavgvamovm0012+cbvartavgvamovm0006+cbvartavgvamovm0003+cbvartavgvamovm0002+cbvartavgvamovm0001)/6')

In [93]:
df10['cbvmobblogvnumvm_avg']  = df.eval('(cbvmobblogvnumvm0024+cbvmobblogvnumvm0012+cbvmobblogvnumvm0006+cbvmobblogvnumvm0003+cbvmobblogvnumvm0002+cbvmobblogvnumvm0001)/6')

In [94]:
df10['cbvmobblogvnumvm24_1'] = df['cbvmobblogvnumvm0024']-df['cbvmobblogvnumvm0001']

In [95]:
df10.iloc[:,-6:] = df10.iloc[:,-6:].fillna(-1)

In [96]:
df10.shape

(1124043, 62)

## 组间特征交叉

### 与月份交叉(按月份分组求平均，中位数或者最大值)

In [97]:
age_mean=df10.groupby('settime_month').mean()['age']

In [98]:
for i in  age_mean.index:   
    df10.loc[df10[df10['settime_month']==i].index,'age_month'] = age_mean[i]

In [99]:
acctage_median=df10.groupby('settime_month').median()['acctage']

In [100]:
for i in  acctage_median.index:   
    df10.loc[df10[df10['settime_month']==i].index,'acctage_month'] = acctage_median[i]

In [101]:
cfvfdpvnumvm0012_max=df10.groupby('settime_month').max()['cfvfdpvnumvm0012']

In [102]:
for i in  cfvfdpvnumvm0012_max.index:   
    df10.loc[df10[df10['settime_month']==i].index,'cfvfdpvnumvm0012_month'] = cfvfdpvnumvm0012_max[i]

In [103]:
covassbalvmaxvm0001_median=df10.groupby('settime_month').median()['covassbalvmaxvm0001'].astype(int)

In [104]:
for i in  covassbalvmaxvm0001_median.index:   
    df10.loc[df10[df10['settime_month']==i].index,'covassbalvmaxvm0001_month'] = covassbalvmaxvm0001_median[i]

In [105]:
covaccvnumvm0024_mean=df10.groupby('settime_month').mean()['covaccvnumvm0024']

In [106]:
for i in  covaccvnumvm0024_mean.index:   
    df10.loc[df10[df10['settime_month']==i].index,'covaccvnumvm0024_month'] = covaccvnumvm0024_mean[i]

### 资产状况与账户数量

In [107]:
df10['Assets_avg1'] = df['covassvhypvm0001']/df['covaccvnumvm0012']

In [108]:
df10['Assets_avg2'] = df['covassbalvmaxvm0001']/df['covaccvnumvm0012']

In [109]:
df10['Assets_avg3'] = df['covasshypvst2vm0012']/df['covaccvnumvm0012']

In [110]:
df10.iloc[:,-3:] = df10.iloc[:,-3:].replace([np.inf, -np.inf,np.NaN], 0)

### 缺失率编码

In [111]:
df10['null_Channel2_transaction'] = df['cbvposdrwvamovm0001']
df10['null_Channel2_transaction'] = df10['null_Channel2_transaction'].apply(lambda x: 0 if x>-1 else 1)

In [112]:
df10['null_Self_service_transaction'] = df['cbvatmtrnovamovm0001']
df10['null_Self_service_transaction'] = df10['null_Self_service_transaction'].apply(lambda x: 0 if x>-1 else 1)

# 分割数据集

In [117]:
df10.shape

(1124043, 72)

In [118]:
X_train = df10[df10['flag'].notnull()]
X_test = df10[df10['flag'].isnull()]
X_test.drop(['cust_id','flag'],axis=1,inplace = True)
y_train = X_train['flag']
X_train.drop(['cust_id','flag'],axis=1,inplace = True)

In [119]:
X_train.shape,X_test.shape

((784000, 70), (340043, 70))

In [120]:
X_train.to_csv('./X_train.csv')
y_train.to_csv('./y_train.csv')
X_test.to_csv("./X_test.csv")#输出到csv文件